# Project Proposal - New York City Property Sales

## Team Members:
1. Aviv Farag - af3228@drexel.edu
2. Abdulaziz Alquzi - aa4472@drexel.edu
3. Jospeh Logan - jrl362@drexel.edu


## Topic - Business Problem Statement

We are data science consultants who are contracted by property management investors in New York City. Their company, supported by investors, wants to buy residential real estate in NYC at the cheapest price possible, renovate, then resell within a year. The renovation analysis is outside the scope of this project, but they want a baseline model that can predict the price of residential real-estate in order to :
 
1. Identify potential undervalued listed properties to buy
2. Predict market price when it’s time to sell in order to sell quickly while maximizing return on investment
 
Because the want to renovate and sell the properties quickly, they want less than 10 residential units, and properties less than 5 million each but are at least ten thousand.


## Dataset Description

`df.info()`:

<center><img  src="img/data_before_cleaning.png" width = "60%" />
<br>
<b><u>Fig. 1</u></b>: Data info before cleaning </center>

### Missing Values

1. *TAX CLASS AT PRESENT* - 738 missing values (single space " ")
1. *BUILDING CLASS AT PRESENT* - 738 missing values (single space " ")
1. *APARTMENT NUMBER* - 65496 missing values (single space " ")<br>
NOTE: Apartment Number shouldn't affect prices, so we can ignore it for our analysis and leave it with missing values.
1. *YEAR BUILT* - 6970 missing values (value is 0)
1. *SALE PRICE* - 14561 missing values (NaN)
1. *LAND SQUARE FEET* - 26252 missing values (NaN)
1. GROSS SQUARE FEET - 27612 missing values (NaN)

### Cleaning Data
Replace price missing values with NaN and drop unneccessary indices:
```
# Replacing missing values with NaN
df.replace(" -  ",np.nan, inplace = True)

# Dropping index and EASE-MENT
df.drop(['EASE-MENT',
         'Unnamed: 0',
         'APARTMENT NUMBER',
         'BUILDING CLASS AT TIME OF SALE'], axis=1, inplace=True)
```

We would like to focus on residential real-estate, and therefore we choose to work on a specific building class categories:
```
# Removing white spaces
df['BUILDING CLASS CATEGORY'] = df['BUILDING CLASS CATEGORY'].str.lstrip()
df['BUILDING CLASS CATEGORY'] = df['BUILDING CLASS CATEGORY'].str.rstrip()

# Categories we would like to focus on 
df1 = df.loc[df['BUILDING CLASS CATEGORY']=='01 ONE FAMILY DWELLINGS']
df2 = df.loc[df['BUILDING CLASS CATEGORY']=='02 TWO FAMILY DWELLINGS']
df3 = df.loc[df['BUILDING CLASS CATEGORY']=='13 CONDOS - ELEVATOR APARTMENTS']
df4 = df.loc[df['BUILDING CLASS CATEGORY']=='10 COOPS - ELEVATOR APARTMENTS']
df5 = df.loc[df['BUILDING CLASS CATEGORY']=='03 THREE FAMILY DWELLINGS']
df6 = df.loc[df['BUILDING CLASS CATEGORY']=='09 COOPS - WALKUP APARTMENTS']
df7 = df.loc[df['BUILDING CLASS CATEGORY']=='15 CONDOS - 2-10 UNIT RESIDENTIAL']
df8 = df.loc[df['BUILDING CLASS CATEGORY']=='07 RENTALS - WALKUP APARTMENTS']
df9 = df.loc[df['BUILDING CLASS CATEGORY']=='08 RENTALS - ELEVATOR APARTMENTS']
df10 = df.loc[df['BUILDING CLASS CATEGORY']=='06 TAX CLASS 1 - OTHER']
df11 = df.loc[df['BUILDING CLASS CATEGORY']=='41 TAX CLASS 4 - OTHER']
df12 = df.loc[df['BUILDING CLASS CATEGORY']=='04 TAX CLASS 1 CONDOS']

# Concat dataframes
applist = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12]
df = pd.concat(applist)
```

Moreover, there were some properties that their sale price is 0, so we filter the rows by selecting the ones with sale price between 10,000 and 5,000,000:
```
# Choose rwos with sale price within our desired range
df = df[df['SALE PRICE'] > 10000]
df = df[df['SALE PRICE'] < 5000000]
```

In addition, we chose to focus on sales in which the residential units is in the range between 1 and 9 (inlcusive) and no commercial units: 
```
# Residential units in range between 1 and 9 inclusive
df = df[df['RESIDENTIAL UNITS'] < 10]
df = df[df['RESIDENTIAL UNITS'] != 0]

# No commercial units 
df = df[df['COMMERCIAL UNITS'] == 0]

# drop unnecessary column
df.drop(labels = ["COMMERCIAL UNITS","TOTAL UNITS"], axis = 1, inplace = True)

```

Finally, we delete all duplicated rows: 
```
# Drop duplicates in our dataframe
df.drop_duplicates(inplace = True)
```




`df.info()`:
<center><img  src="img/info_after_cleaning.png" width = "60%" />
<br>
<b><u>Fig. 2</u></b>: Data info after cleaning</center>



### Initial  Feature Analysis:

 
1. **Borough** is the borough in which the unit is located: '1':'Manhattan', '2':'Bronx', '3': 'Brooklyn', '4':'Queens','5':'Staten Island'. Location is a key feature in real estate, and this is especially true in NYC. For the purposes of exploratory analysis, we have converted the numeric values into their proper names.  Below you will see a clear distinction in price by neighborhood, with Manhattan being much more expensive. For constructing the model we will likely one hot encode.<br><br>
 
1. **Neighborhood** is the specific neighborhood within the borough. There is a strong relationship with neighborhood and sale price. Much like borough, because the neighborhoods are not explicitly ranked, a one hot coding strategy will likely be used.<br><br>
 
1. **Building Class Category** is an important feature as it separates between attached and unattached houses or elevator apartments.<br><br>
 
1. **Tax class a present** is the tax class and is heavily correlated with both sale price and tax class at time of sale. Tax class and number of units as correlated as the tax class depends on how may units. There is a big risk of data leakage with this feature. The models real world success would depend on accurately determining the tax class before selling or purchasing a property. Because of this, we may want to remove this feature.
[Source](https://blocksandlots.com/property-taxes-in-nyc-all-you-need-to-know/)<br><br>

1. **Block and Lot** The combination of borough, block, and lot forms a unique key for property in New York City. Commonly called a BBL
[Source](https://blocksandlots.com/property-taxes-in-nyc-all-you-need-to-know/)<br><br>
 
1. **Building Class at present** <br><br>
According to [nyc.gov](https://www1.nyc.gov/assets/finance/downloads/pdf/07pdf/glossary_rsf071607.pdf): <br>
“The Building Classification is used to describe a property’s constructive use. The first position of the Building Class is a letter that is used to describe a general class of properties (for example “A” signifies one-family homes, “O” signifies office buildings. “R” signifies condominiums). The second position, a number, adds more specific information about the property’s use or construction style (using our previous examples “A0” is a Cape Cod style one family home, “O4” is a tower type office building and “R5” is a commercial condominium unit). The term Building Class used by the Department of Finance is interchangeable with the term Building Code used by the Department of Buildings.”
<br><br>
Because this feature has direct overlap with many other features, we will likely remove it.<br><br>

1. **Address** Is the actual address. Because of the variance we will remove this feature. However, it could be potentially used to crosswalk longitude and latitude, but that would require an additional dataset.<br><br>
 
1. **Zip Code** Zip codes are difficult to work with in machine learning problems because it’s an integer and a higher or lower zip code won’t necessarily mean it’s better or worse. If we are going to use it we will one hot encode.<br><br>

1. **Residential Units** are the number of residential units for sale. This is correlated with price as 8 units will likely cost more than 2 in a similar neighborhood. For exploratory analysis we examine the price per unit.<br><br>
 
1. **Land Square Feet** is the land area of the property. This is a valuable feature for predicting price.<br><br>
 
1. **Gross Square Feet** According to nyc.gov: “The total area of all the floors of a building as measured from the exterior surfaces of the outside walls of the building, including the land area and space within any building or structure on the property.”  This is also a valuable feature for predicting price. However, it’s important for compare between this and location as a smaller property in the center of Manhattan, may be more expensive than a much larger property in Staten Island.<br><br>
 
1. **Year Built** is the year the structure was built. Many of the properties were built a long time ago, but it’s worth further testing this feature before elimination. 
Tax class at time of sale: See above for tax class. It will be difficult to accurately predict this and it has a very high risk of data leakage. This feature will almost certainly be removed.<br><br>

1. **Tax Class at Time of Sale** - <br> 
  &nbsp;[Property Tax Rate](https://www1.nyc.gov/site/finance/taxes/property-tax-rates.page "Click here to move to NYC property tax rates") for 2016/2017 was:
    - Class 1 - 19.991\%	
    - Class 2 - 12.892\%	
    - Class 3 - 10.934\%	
    - Class 4 - 10.574\% <br><br>
        **Note**:This feature might cause data leakage due to its correlation with SALE PRICE (higher price -> higher tax), and therefore we will not use it in our machine learning algorithm.<br><br>

1. **Sale Price** is our target variable. Due to the scope of the business problem we are limiting the dataset to 10,000 and 5,000,0000.<br><br>
 
1. **Sale Date** Is the date of the sale. We may want to look at the sale month to determine if we can purchase a property in a slower month for real estate i.e. buy in the winter cheaply and resell in a hotter month like this the spring.<br>


## Exploratory Data Analysis (EDA)



### Residential Units vs. Sale Price

We used Seaborn package to plot a boxplot of residential units versus sale price. 

```
plt.figure(figsize=(10,6))
sns.boxplot(x='RESIDENTIAL UNITS', y='SALE PRICE', data=df)
plt.title('Residential Units vs Sale Price')
plt.show()
```

<center><img  src="img/res_vs_saleprice.png" width = "70%" />
<br>
<b><u>Fig. 3</u></b>: Box plot of Residential Units and Sale Price </center>

The mean SALE PRICE of 8 residential units is higher than the mean value of any other number of residential units. Also, mean SALE PRICE increases as the residential units increase up to 8 residential units. Then, it goes down for 9 residential units. Moreover, there are a lot of outliers in 1,2,3, and 4 residential units which could be due to Borough or other parameters that were not taken into account in this plot.


### Borough vs. Sale Price

```
plt.figure(figsize=(10,6))
sns.boxplot(x='BOROUGH', y='SALE PRICE', data=df)
plt.title('Borough vs Sale Price')
plt.show()
```


<center><img  src="img/borough_vs_saleprice.png" width = "70%" />
<br>
<b><u>Fig. 3</u></b>: Borough vs Sale Price </center>

Figure 3 presents the distribution of Sale price among Borough. The mean SALE PRICE in Manhattan is the highest. Also, the distribution of SALE PRICE in Manhattan is the longest (up to ~4.5 million without outliers).

### Building Class Category vs Price Per unit

We made a new column called price per unit which is the SALE PRICE divided by the number of residential units in order to standardize the data for the next plots. 

```
plt.figure(figsize=(20,6))
ax = sns.boxplot(x='BUILDING CLASS CATEGORY', y='price per unit', data=df)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
plt.title('Building Class Category vs Price Per Unit')
plt.show()
```

<center><img  src="img/building_category_vs_price_per_unit.png" width = "100%" />
<br>
<b><u>Fig. 4</u></b>: Building Class Category versus Price Per Unit </center>

Across the city, one family dwellings per unit are generally more expensive than two or three faimly dwllings.
However, many of the condos are on average more expensive than one family dwellings. This is not taking into account Borough
For example, a condo in Manhattan may cost more than a one family dwelling in Staten island. 
<br><br> 

```
plt.figure(figsize=(20,8))
ax = sns.barplot(x='BUILDING CLASS CATEGORY', y='price per unit',hue='BOROUGH', data=df)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
plt.legend(loc='upper right',fontsize=14)
plt.title('Price Per Unit vs Building Class Category in each Borough')
plt.show()
```


<center><img  src="img/price_vs_bc_&_brorough.png" width = "100%" />
<br>
<b><u>Fig. 5</u></b>: Price Per Unit vs Building Class Category & Borough </center>

It may also be fair to create another category of Manhattan = 1 and the rest 0. We can further use neighborhood to make this determination.
<br><br> 

```
manhattendf = df.loc[df['BOROUGH']=='Manhattan']
manhattendfn = manhattendf.groupby('NEIGHBORHOOD')['price per unit'].median().reset_index()
manhattendfn.rename(columns = {'price per unit':'Median Price Per Unit'}, inplace = True)
manhattendfn = manhattendfn.sort_values(by='Median Price Per Unit', ascending=False)
manhattendfn = manhattendfn.dropna()
plt.figure(figsize=(25,8))
ax = sns.barplot(x='NEIGHBORHOOD', y='Median Price Per Unit', data =manhattendfn )
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.title('Median Price Per Unit vs. Neighborhood in Manhattan', fontsize = 18)
plt.xlabel("Neighborhood in Manhattan", fontsize = 18)
plt.ylabel("Median Price Per Unit", fontsize = 18)
plt.rcParams["xtick.labelsize"] = 12
plt.rcParams["ytick.labelsize"] = 12
plt.show()
```

<center><img  src="img/manhattan_ppu.png" width = "100%" />
<br>
<b><u>Fig. 6</u></b>: Median Price Per Unit distribution among neighborhoods in Manhattan </center>



In figure 6, we can clearly see the impact of neighborhood (Location of the property) on the Median Price Per Unit. A property on either Civic Center or SOHO is more likley to be sold in a higher price than a property in INWOOD. 

## Machine Learning Algorithms

Accurately predicting the sales price is a regression problem. Because of this we will be creating 3 regression models:
 
* Linear Regression
 
* Decision Tree
  
* Random Forrest Regressor
 
We will evaluate the model’s performance with root mean square error and mean absolute error.

## Source

Dataset was found at [Kaggle](https://www.kaggle.com/new-york-city/nyc-property-sales). <br>
The origin of the data in this dataset is NYC Department of Finance [Rolling Sales](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) 